# 삼성전자 ML 이해 및 구현 파이널 프로젝트

> - 목적
>> 건축물에 사용되는 콘크리트의 압축강도를 예측하기 위한 ML모델을 생성합니다.  
>> 프로젝트 목적에 따라 올바른 ML모델을 선택하고 파라메터 서칭을 통해 우수한 모델을 구축합니다.  
>> 콘크리트 혼합물 및 연식에 따른 콘크리트의 내구성을 예측하여 건물의 유지보수 시 참고할 수 있는 모델을 생성함에 목적이 있습니다.

> - 진행방식
>> 아래 굵은 글씨로 작성 되어있는 테스트를 수행하며 테스크 아래 수행 결과값과 비교하며 프로젝트를 진행합니다.  
>> 코드 실행 결과값이 없을 경우 # 결과값 없음, 코드 실행 결과가 결과값이랑 다를 수 있는 경우 # 결과값 다를 수 있음으로 가늠합니다.  
>> 필요 시 추가적으로 셀을 생성하며 진행합니다.

# 필요모듈 import

In [2]:
# CODE HERE
from itertools import product
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix


# LinearRegression, DecisionTreeRegressor, RandomForestRegressor, XGBRegressor


# 데이터 로딩 및 확인
## 함께 제공 된 concrete.csv 파일을 읽어들여 데이터프레임으로 저장 후 첫 5개 샘플 데이터 확인

In [3]:
# CODE HERE
df = pd.read_csv('concrete.csv')
df.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,Concretecompressivestrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


위 데이터는 콘크리트 압축강도를 예측하기 위한 데이터셋으로 콘크리트 제작 시 사용한 혼합재료 및 연령을 feature로 수집한 데이터 입니다.  
* 각 컬럼 데이터에 대한 설명은 아래와 같습니다.

| columns name | 의미 | 단위 |
|-|-|-|
| Cement | 시멘트  | kg/m^3 |
| BlastFurnaceSlag | 용광로슬래그 | kg/m^3 |
| FlyAsh | 비산회 | kg/m^3 |
| Water | 물 | kg/m^3 |
| Superplasticizer | 고성능가소제 | kg/m^3 |
| CoarseAggregate | 조골재 | kg/m^3 |
| FineAggregate | 미세골재 | kg/m^3 |
| Age | 연령 | 일(day) |
| Concretecompressivestrength | 콘크리트압축강도 | MPa |

## 데이터 shape 확인

In [4]:
# CODE HERE
df.shape

(1030, 9)

## info() 메소드로 데이터의 대략적인 정보 확인

In [5]:
# CODE HERE
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Cement                       1030 non-null   float64
 1   BlastFurnaceSlag             1030 non-null   float64
 2   FlyAsh                       1030 non-null   float64
 3   Water                        1030 non-null   float64
 4   Superplasticizer             1030 non-null   float64
 5   CoarseAggregate              1030 non-null   float64
 6   FineAggregate                1030 non-null   float64
 7   Age                          1030 non-null   int64  
 8   Concretecompressivestrength  1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


결측데이터가 존재하지 않고 데이터 타입 또한 float, int형 데이터이기에 ML모델 학습을 위한 전처리 과정이 추가로 필요하지 않을 것 같습니다.  
데이터 분할 과정을 바로 수행하겠습니다.

## 타겟데이터 분할
**콘크리트압축강도를 예측하기 위한 모델링을 위해 타겟 데이터를 분할 합니다.**

In [6]:
# CODE HERE
y = df['Concretecompressivestrength']
X = df.drop(['Concretecompressivestrength'], axis=1)
X

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28


In [7]:
# 출력 결과값 없음

## 학습데이터 분할
**test 데이터셋의 비율을 25%, random_state=42 로 동일한 데이터가 학습 될 수 있도록 데이터 분할을 수행합니다.**  
**데이터 분할 후 생성 된 X_train, X_test, y_train, y_test 데이터 셋의 shape을 순서대로 확인합니다.**

In [7]:
# CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((772, 8), (258, 8), (772,), (258,))

## 검증데이터 셋 생성
**비교우위에 있는 모델의 default 성능 비교를 위해 X_train, y_train 데이터를 분할하여 validation set 생성을 수행합니다.**  
**train_test_split 함수에 X_train, y_train 데이터를 전달하여 생성하며 검증셋의 비율을 30%, random_state=42 설정하여 작업합니다.**  
**작업이후 X_train2, X_val, y_train2, y_val 데이터셋의 shape을 순서대로 확인합니다.**

In [8]:
# CODE HERE
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
X_train2.shape, X_val.shape, y_train2.shape, y_val.shape


((540, 8), (232, 8), (540,), (232,))

## 모델 서칭
**생성한 validation set을 사용하여 LinearRegression, DecisionTreeRegressor, RandomForestRegressor, XGBRegressor 모델의 기본성능을 비교하고 default성능이 가장 우수한 모델을 선정 해봅니다.**  
**모델 서칭 이후 해당 모델의 r2 score 와 rmse를 출력해봅시다.**

In [9]:
# CODE HERE
# 아래에 자유롭게 해당 작업 코드를 작성하시면 됩니다.

# LinearRegression, DecisionTreeRegressor, RandomForestRegressor, XGBRegressor
from sklearn.metrics import root_mean_squared_error, r2_score



lr = LinearRegression()
dtr = DecisionTreeRegressor(random_state=42)
rfr = RandomForestRegressor(random_state=42)
xgr = XGBRegressor(random_state=42)




# lr.fit(X_train2, y_train2)
# lr_pred = lr.predict(X_val)
# lr_r2 = r2_score(y_val, lr_pred)
# lr_rmse = root_mean_squared_error(y_val, lr_pred)
# print(lr_r2, lr_rmse)

# dtr.fit(X_train2, y_train2)
# dtr_pred = dtr.predict(X_val)
# dtr_rmse = root_mean_squared_error(y_val, dtr_pred)
# dtr_r2 = r2_score(y_val, dtr_pred)
# print(dtr_r2, dtr_rmse)

# rfr.fit(X_train2, y_train2)
# rfr_pred = rfr.predict(X_val)
# rfr_rmse = root_mean_squared_error(y_val, rfr_pred)
# rfr_r2 = r2_score(y_val, rfr_pred)
# print(rfr_r2, rfr_rmse)

# xgr.fit(X_train2, y_train2)
# xgr_pred = xgr.predict(X_val)
# xgr_rmse = root_mean_squared_error(y_val, xgr_pred)
# xgr_r2 = r2_score(y_val, xgr_pred)
# print(xgr_r2, xgr_rmse)


model_list = [lr, dtr, rfr, xgr]

best_r2 = 0
best_rmse = 99999
best_model_r2 = None
best_model_rmse = None

for est in model_list:
    est.fit(X_train2, y_train2)
    est_pred = est.predict(X_val)
    est_rmse = root_mean_squared_error(y_val, est_pred)
    est_r2 = r2_score(y_val, est_pred)
    if est_rmse < best_rmse :
        best_rmse = est_rmse
        best_model_rmse = est
    if est_r2 > best_r2 :
        best_r2 = est_r2
        best_model_r2 = est

print(best_model_r2, best_model_rmse)
print(best_r2, best_rmse)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...) XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, f

In [11]:
# 출력 결과값 다를 수 있음


위에서 확인한 모델이 개선을 위해 비교대상이 되는 베이스라인 모델입니다. 이후 과정에서는 이 베이스라인보다 더 우수한 모델을 생성함을 목표로 합니다.

# 파라메터 서칭
베이스라인 모델의 파라메터 서칭을 통해 더 우수한 성능의 모델을 구현합니다.  
## GridSearchCV를 사용하기 위한 모델 생성 및 파라메터 그리드 생성을 해 봅시다.

In [10]:
# CODE HERE
# 아래에 자유롭게 해당 작업 코드를 작성하시면 됩니다.
# 파라메터 서칭을 여러번 하시더라도 서칭 하신 그리드의 히스토리는 주석으로 남겨주세요



# import pandas as pd
# import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_error
# from sklearn.pipeline import Pipeline




# XGBoost 모델 및 파라미터 그리드 설정
xgr = XGBRegressor(random_state=42)
# params ={
#     'max_depth' : [3, 4, 5],
#     'subsample' : [0.7, 0.8, 0.9],
#     'colsample_bytree' : [0.6, 0.7, 0.8, 0.9],
#     'n_estimators' : [200, 300, 400, 500, 600],
#     'learning_rate' : [0.1]
# }


# params ={
#     'max_depth' : [2, 3, 4],
#     'subsample' : [0.75, 0.8, 0.85],
#     'colsample_bytree' : [0.6, 0.65, 0.7, 0.75, 0.8],
#     'n_estimators' : [550, 600, 650, 700, 750],
#     'learning_rate' : [0.1]
# }


# params ={
#     'max_depth' : [3],
#     'subsample' : [0.825, 0.85, 0.875, 0.9],
#     'colsample_bytree' : [0.63, 0.64, 0.65, 0.66, 0.67],
#     'n_estimators' : [630, 640, 650, 660, 670],
#     'learning_rate' : [0.1]
# }

params ={
    'max_depth' : [3],
    'subsample' : [0.84, 0.85, 0.86],
    'colsample_bytree' : [0.62, 0.625, 0.63, 0.635],
    'n_estimators' : [670, 675, 680, 685],
    'learning_rate' : [0.1]
}



# GridSearchCV 설정 및 실행

grid = GridSearchCV(estimator=xgr,
                    param_grid=params,
                    scoring='neg_mean_squared_error',
                    verbose=2,
                    n_jobs=-1,
                    cv=5)





In [ ]:
# 출력 결과값 없음

## GridSearchCV 모델 생성
**GridSearchCV를 사용하여 파라메터 서칭을 진행합니다. 평가지표는 모델링 목적에 맞게 rmse를 사용하여 모델을 개선합니다.**  
**추가적으로 5fold cross validation을 수행할 수 있도록 파라메터 설정을 해주세요**  
**파라메터 조합을 딥하게 서칭하면 학습 시간이 오래 소요됩니다. 부분적으로 생성 한 범위의 조합을 여러번 학습하여 서칭하심을 추천드립니다.**

In [11]:
# CODE HERE
grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'colsample_bytree': 0.625, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 680, 'subsample': 0.85}


In [ ]:
# 출력 결과값 숨김

## 학습이 완료 된 모델에 test데이터를 적용하여 예측값을 생성한 후 r2 score, rmse를 출력합니다.

In [12]:
# CODE HERE
# 최적의 모델로 예측
# 0.912471552160593 4.94134263842851

# RMSE 계산
rmse = root_mean_squared_error(y_test, y_pred)
r2_ = r2_score(y_test, y_pred)

print(f"Best parameters: {grid.best_params_}")
print(f"RMSE: {rmse}")
print(f"r2: {r2_}")

Best parameters: {'colsample_bytree': 0.625, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 680, 'subsample': 0.85}
RMSE: 3.9025166941275256
r2: 0.9437759414452838


In [148]:
# 출력 결과값 다를 수 있음
# 위에서 선정한 베이스라인 결과값 보다 좋은 스코어를 보이는 파라메터를 찾아주시면 되겠습니다.


BEST R2 score : 0.9403917642647227
BEST RMSE : 4.018248649895231


## Polynominal Features 적용
위에서 생성한 모델의 예측성능도 뛰어나지만 추가적으로 적용할 수 있는 전처리 방법을 사용하여 모델의 성능을 조금 더 끌어올려보겠습니다.  
**PolyminalFeatures 전처리 모델을 X_train, X_test 데이터에 적용하여 2차항에 해당하는 파생변수를 생성합니다.**  
**작업이후 변환 된 X_train, X_test 데이터의 shape을 순서대로 확인합니다.**

In [13]:
# CODE HERE
# 필요모델 import
from sklearn.preprocessing import PolynomialFeatures

polyf = PolynomialFeatures(degree=2, include_bias=False)

'''
파라메터
degree=2 : 차수설정
include_bias=False : 상수항 제거(필수)
'''


# X_train에 해당하는 변수를 학습과 동시에 적용
polyf.fit(X_train)  #머신 러닝 모델이 아니기에 정답 필요 없음. 변환 대상 데이터만 학습


polyf.transform(X_train)

# 변환식 반환
polyf.get_feature_names_out()

# 데이터프레임으로 제작 후 데이터 확인
poly_X_train = pd.DataFrame(polyf.transform(X_train), columns=polyf.get_feature_names_out())



poly_X_test = pd.DataFrame(polyf.transform(X_test), columns=polyf.get_feature_names_out())

poly_X_train.shape, poly_X_test.shape



((772, 44), (258, 44))

((772, 44), (258, 44))

## Polynominal Features GridSearchCV
**위에서 생성한 다항변수를 사용하여 그리드 서치를 다시 수행합니다.**

In [14]:
# CODE HERE
# XGBoost 모델 및 파라미터 그리드 설정
xgr = XGBRegressor(random_state=42)


# params ={
#     'max_depth' : [3],
#     'subsample' : [0.84, 0.85, 0.86],
#     'colsample_bytree' : [0.62, 0.625, 0.63, 0.635],
#     'n_estimators' : [670, 675, 680, 685],
#     'learning_rate' : [0.1]
# }

# params ={
#     'max_depth' : [3],
#     'subsample' : [0.845, 0.85, 0.855],
#     'colsample_bytree' : [0.605, 0.61, 0.615, 0.62, 0.625],
#     'n_estimators' : [671, 673, 675, 677],
#     'learning_rate' : [0.1]
# }

# params ={
#     'max_depth' : [3],
#     'subsample' : [0.849, 0.85, 0.851],
#     'colsample_bytree' : [0.6145, 0.615, 0.6155],
#     'n_estimators' : [674, 675, 676],
#     'learning_rate' : [0.1]
# }

# RMSE: 3.9364251626709157
# r2: 0.9427946494148012



# params ={
#     'max_depth' : [2, 3, 4, 5],
#     'subsample' : [0.85],
#     'colsample_bytree' : [0.6145, 0.615],
#     'n_estimators' : [675],
#     'learning_rate' : [0.05, 0.1, 0.15]
# }


params ={
    'max_depth' : [3],
    'subsample' : [0.8495, 0.85, 0.8505],
    'colsample_bytree' : [0.6144, 0.6145, 0.6146],
    'n_estimators' : [675],
    'learning_rate' : [0.1]
}


# GridSearchCV 설정 및 실행

grid = GridSearchCV(estimator=xgr,
                    param_grid=params,
                    scoring='neg_mean_squared_error',
                    verbose=2,
                    n_jobs=-1,
                    cv=5)


grid.fit(poly_X_train, y_train)
print("Best parameters:", grid.best_params_)
best_model = grid.best_estimator_
y_pred = best_model.predict(poly_X_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'colsample_bytree': 0.6144, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 675, 'subsample': 0.8505}


In [ ]:
# 출력 결과값 숨김

## best parameter 확인 및 r2 score, rmse를 출력합니다.

In [15]:
# CODE HERE
# RMSE 계산
rmse = root_mean_squared_error(y_test, y_pred)
r2_ = r2_score(y_test, y_pred)

# print(f"Best parameters: {grid.best_params_}")
print(f"RMSE: {rmse}")
print(f"r2: {r2_}")

# 기존 결과
# RMSE: 3.9025166941275256
# r2: 0.9437759414452838

RMSE: 3.8873536845309533
r2: 0.9442120034971683


In [150]:
# best_param 출력결과 숨김
# 출력 결과값 다를 수 있음


BEST R2 score : 0.9447297174951115
BEST RMSE : 3.8692742728797382


# 수고하셨습니다!!